<a href="https://colab.research.google.com/github/andrewli4938/TrafficLightDetection/blob/main/Group1_Traffic_Light_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.5/898.5 kB 18.9 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn
import pandas as pd
from sklearn.model_selection import train_test_split
import os
import glob
import shutil
from PIL import Image
from google.colab import drive

from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
# import our dataset
import kagglehub
source_path = kagglehub.dataset_download("mbornoe/lisa-traffic-light-dataset")

100%|██████████| 4.21G/4.21G [00:27<00:00, 166MB/s]

Extracting files...


In [4]:
%pwd
%rm -r data
%mkdir data  # root folder for data

%cd data
%mkdir images
%mkdir labels

%cd images
%mkdir train
%mkdir val
%mkdir test

%cd ../labels
%mkdir train
%mkdir val
%mkdir test

%cd /content

rm: cannot remove 'data': No such file or directory
/content/data
/content/data/images
/content/data/labels
/content


In [5]:
%mkdir /content/data/images/day
%mkdir /content/data/images/night

%mkdir /content/data/labels/day
%mkdir /content/data/labels/night

In [6]:
# these two only have total 1000 images, and have duplicate filenames so we can deal with them later
%mv {source_path}/sample-dayClip6 {source_path}/..
%mv {source_path}/sample-nightClip1 {source_path}/..

In [7]:
# move all of the filenames into list
image_paths = glob.glob(f"{source_path}/**/*.jpg", recursive=True)

# process the annotations
target = "frameAnnotationsBOX.csv"
annotations = pd.DataFrame()
annotation_paths = glob.glob(f"{source_path}/**/{target}", recursive=True)
for p in annotation_paths:
  # combine the csv files into one
  new_frame = pd.read_csv(p, sep=";")
  new_frame = new_frame.drop(['Origin file', 'Origin frame number', 'Origin track', 'Origin track frame number'], axis=1)
  annotations = pd.concat([annotations, new_frame])
filenames = annotations[annotations.columns[0]].str.split("/").str[-1]
annotations["Filename"] = filenames
annotations.set_index("Filename", inplace=True)
annotations

,Annotation tag,Upper left corner X,Upper left corner Y,Lower right corner X,Lower right corner Y
Filename,,,,,
nightSequence1--00000.jpg,go,667,385,676,400
nightSequence1--00000.jpg,stop,520,387,532,408
nightSequence1--00000.jpg,stop,725,385,740,412
nightSequence1--00001.jpg,go,667,385,676,400
nightSequence1--00001.jpg,stop,520,386,532,407
...,...,...,...,...,...
dayClip8--01033.jpg,go,923,270,977,330
dayClip8--01034.jpg,go,982,250,1039,325
dayClip8--01035.jpg,go,1043,244,1106,324


In [8]:
def move_files(paths_list, destination_dir):
  for file in paths_list:
    shutil.copy(file, destination_dir)

In [9]:
# TODO: split the data into train, val, test
image_train, image_val_test = train_test_split(image_paths, train_size=0.7, test_size=0.3, random_state=42, shuffle=True)
image_val, image_test = train_test_split(image_val_test, train_size=0.5, random_state=42, shuffle=True)

print(len(image_train))
print(len(image_val))
print(len(image_test))

image_train_dir = "/content/data/images/train"
image_val_dir = "/content/data/images/val"
image_test_dir = "/content/data/images/test"
move_files(image_train, image_train_dir)
move_files(image_val, image_val_dir)
move_files(image_test, image_test_dir)

print(len(os.listdir(image_train_dir)))
print(len(os.listdir(image_val_dir)))
print(len(os.listdir(image_test_dir)))

30111
6452
6453
30111
6452
6453


In [10]:
# import yaml file: defines image locations and encodes classes into numbers for the model
!wget https://raw.githubusercontent.com/andrewli4938/TrafficLightDetection/refs/heads/main/data.yaml

--2024-12-02 21:57:06--  https://raw.githubusercontent.com/andrewli4938/TrafficLightDetection/refs/heads/main/data.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 177 [text/plain]
Saving to: ‘data.yaml’

data.yaml           100%[===================>]     177  --.-KB/s    in 0s      

2024-12-02 21:57:07 (9.70 MB/s) - ‘data.yaml’ saved [177/177]



In [11]:
def convert_to_xywh(box_coordinates, image_dimensions):
  image_width, image_height = image_dimensions
  x1, y1, x2, y2 = box_coordinates
  width, height = x2-x1, y2-y1
  x_center = (x1+x2)/2
  y_center = (y1+y2)/2

  x_center = x_center/image_width
  y_center = y_center/image_height
  width = width/image_width
  height = height/image_height

  return (x_center, y_center, width, height)

In [12]:
"""
The *.txt file should be formatted with one row per object in
class x_center y_center width height format. Box coordinates
must be in normalized xywh format (from 0 to 1). If your boxes
are in pixels, you should divide x_center and width by image width,
and y_center and height by image height. Class numbers should be
zero-indexed (start with 0).
"""
image_dims = (1280, 960)

def create_labels(source_dir, destination_dir, annotations_df, encoding):
  for file in os.listdir(source_dir):
    write_path = os.path.join(destination_dir, file.replace(".jpg", ".txt"))
    if file in annotations_df.index:
      rows = annotations_df.loc[file]
      if isinstance(rows, pd.Series):
        rows = pd.DataFrame([rows])
    else:
      continue
    with open(write_path, "w") as image_label:
      for i in range(len(rows)):  # iterate through each label for current image
        class_number = encoding[rows.iloc[i, 0]]
        bounding_box = rows.iloc[i, 1:]
        # print(bounding_box)
        # print(f"image is: {file} at index {i}")
        # print(type(rows))
        xywh = convert_to_xywh(bounding_box, image_dims)  # calculate bounding box coordinates here
        image_label.write(f"{class_number} {xywh[0]} {xywh[1]} {xywh[2]} {xywh[3]}\n")


In [13]:
class_encoding = {"stop": 0, "stopLeft": 1, "warning": 2, "warningLeft": 3,
                  "warningLeft": 4, "go": 5, "goForward": 6, "goLeft": 6}

label_train_dir = "/content/data/labels/train"
label_val_dir = "/content/data/labels/val"
label_test_dir = "/content/data/labels/test"

create_labels(image_train_dir, label_train_dir, annotations, class_encoding)
create_labels(image_val_dir, label_val_dir, annotations, class_encoding)
create_labels(image_test_dir, label_test_dir, annotations, class_encoding)

print(len(os.listdir(label_train_dir)))
print(len(os.listdir(label_val_dir)))
print(len(os.listdir(label_test_dir)))

25396
5444
5425


In [31]:
sample_path = "/root/.cache/kagglehub/datasets/mbornoe/lisa-traffic-light-dataset/versions"

In [42]:
annotation_path = f"{sample_path}/sample-dayClip6/sample-dayClip6/frameAnnotationsBOX.csv"
day_annotations = pd.read_csv(annotation_path, sep=";")
day_annotations.set_index("Filename", inplace=True)
day_annotations = day_annotations.iloc[:, :5]

In [43]:
annotation_path = f"{source_path}/../sample-nightClip1/sample-nightClip1/frameAnnotationsBOX.csv"
night_annotations = pd.read_csv(annotation_path, sep=";")
night_annotations.set_index("Filename", inplace=True)
night_annotations = night_annotations.iloc[:, :5]

In [36]:
!cp {sample_path}/sample-dayClip6/sample-dayClip6/frames/* /content/data/images/day/
!cp {sample_path}/sample-nightClip1/sample-nightClip1/frames/* /content/data/images/night/

In [44]:
image_day_dir = "/content/data/images/day"
image_night_dir = "/content/data/images/night"

create_labels(image_day_dir, "/content/data/labels/day", day_annotations, class_encoding)
create_labels(image_night_dir, "/content/data/labels/night", night_annotations, class_encoding)

In [5]:
torch.cuda.is_available()

True

In [6]:
torch.cuda.empty_cache()

In [7]:
!nvidia-smi

Mon Dec  2 07:49:45 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P0              48W / 400W |   6072MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [14]:
!rm -r /content/runs/detect/train2

In [21]:
shutil.make_archive(base_name="final", format="zip", root_dir="/content/runs")

'/content/final.zip'

In [ ]:
# model training
model = YOLO("yolo11n.pt")
results = model.train(data="/content/data.yaml", epochs=20, imgsz=896, batch=128)

Ultralytics 8.3.39 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/content/data.yaml, epochs=20, time=None, patience=100, batch=128, imgsz=896, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, sho

train: Scanning /content/data/labels/train... 25396 images, 4715 backgrounds, 0 corrupt: 100%|██████████| 30111/30111 [00:22<00:00, 1319.86it/s]

train: WARNING ⚠️ /content/data/images/train/nightClip4--00378.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/data/images/train/nightClip4--00380.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/data/images/train/nightClip4--00383.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/data/images/train/nightClip4--00384.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/data/images/train/nightClip4--00385.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/data/images/train/nightClip4--00386.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/data/images/train/nightClip4--00387.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/data/images/train/nightClip4--00388.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/data/images/train/nightClip4--00389.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/data/images/train/nightClip4--00390.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/data/images/train/night

train: New cache created: /content/data/labels/train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
val: Scanning /content/data/labels/val.cache... 5444 images, 1008 backgrounds, 0 corrupt: 100%|██████████| 6452/6452 [00:00<?, ?it/s]

...

      20/20      37.1G     0.7292     0.3778     0.8442         65        896: 100%|██████████| 236/236 [02:47<00:00,  1.41it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 26/26 [00:24<00:00,  1.06it/s]

                   all       6452      16263      0.891      0.869      0.911       0.63



20 epochs completed in 1.119 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 5.5MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 5.5MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics 8.3.39 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
YOLO11n summary (fused): 238 layers, 2,583,517 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 26/26 [00:29<00:00,  1.13s/it]


                   all       6452      16263      0.893       0.87      0.912      0.631
                  stop       2792       6561      0.957      0.934      0.975      0.763
              stopLeft       1565       1939      0.968      0.908       0.97      0.832
               warning        140        331      0.932        0.9      0.951      0.577
                    go         25         42       0.68       0.69      0.674      0.328
             goForward       2788       6954       0.94       0.87      0.955      0.695
                goLeft        324        436      0.883      0.916      0.947      0.591
Speed: 0.2ms preprocess, 0.6ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/detect/train2


In [14]:
model = YOLO("/content/last.pt")
results = model.train(data="/content/data.yaml", epochs=30, imgsz=896, batch=128, augment=True, conf=0.5)

Ultralytics 8.3.39 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
engine/trainer: task=detect, mode=train, model=/content/last.pt, data=/content/data.yaml, epochs=30, time=None, patience=100, batch=128, imgsz=896, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=0.5, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True,

train: Scanning /content/data/labels/train.cache... 25396 images, 4715 backgrounds, 0 corrupt: 100%|██████████| 30111/30111 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/data/images/train/nightClip4--00378.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/data/images/train/nightClip4--00380.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/data/images/train/nightClip4--00383.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/data/images/train/nightClip4--00384.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/data/images/train/nightClip4--00385.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/data/images/train/nightClip4--00386.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/data/images/train/nightClip4--00387.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/data/images/train/nightClip4--00388.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/data/images/train/nightClip4--00389.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/data/images/train/nightClip4--00390.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/data/images/train/night

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Scanning /content/data/labels/val.cache... 5444 images, 1008 backgrounds, 0 corrupt: 100%|██████████| 6452/6452 [00:00<?, ?it/s]

...

      30/30      39.6G     0.6292     0.3121     0.8248         71        896: 100%|██████████| 236/236 [02:49<00:00,  1.39it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 26/26 [00:23<00:00,  1.13it/s]

                   all       6452      16263      0.957      0.835      0.898      0.675



30 epochs completed in 1.678 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 5.5MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 5.5MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics 8.3.39 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
YOLO11n summary (fused): 238 layers, 2,583,517 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 26/26 [00:35<00:00,  1.36s/it]


                   all       6452      16263      0.934      0.865      0.909      0.674
                  stop       2792       6561      0.949      0.947      0.966       0.81
              stopLeft       1565       1939      0.967      0.929       0.96       0.87
               warning        140        331      0.956      0.924      0.953      0.618
                    go         25         42        0.9      0.643      0.739      0.372
             goForward       2788       6954      0.928       0.89       0.93      0.739
                goLeft        324        436      0.904       0.86      0.905      0.636
Speed: 0.2ms preprocess, 1.7ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/detect/train2


In [36]:
val_model = YOLO('/content/best.pt')
val_result = val_model.val(data='/content/data.yaml', split="test")

Ultralytics 8.3.40 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLO11n summary (fused): 238 layers, 2,583,517 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning /content/data/labels/test.cache... 5425 images, 1028 backgrounds, 0 corrupt: 100%|██████████| 6453/6453 [00:00<?, ?it/s]

val: WARNING ⚠️ /content/data/images/test/nightClip4--00382.jpg: 1 duplicate labels removed



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 404/404 [00:56<00:00,  7.14it/s]


                   all       6453      16466      0.929      0.921      0.958      0.705
                  stop       2814       6811      0.962      0.949       0.98      0.817
              stopLeft       1520       1854      0.971      0.931      0.981       0.88
               warning        159        385       0.94      0.906      0.957      0.601
                    go         36         52      0.848      0.904        0.9      0.547
             goForward       2768       6980      0.931      0.905      0.964      0.738
                goLeft        283        384       0.92       0.93      0.968      0.644
Speed: 0.3ms preprocess, 5.3ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs/detect/val10


In [37]:
val_result.results_dict

{'metrics/precision(B)': 0.9286623477680597,
 'metrics/recall(B)': 0.9208902777649394,
 'metrics/mAP50(B)': 0.95844630489005,
 'metrics/mAP50-95(B)': 0.7045648796023997,
 'fitness': 0.7299530221311648}

In [ ]:
#TODO: the label files are producing duplicate files, fix the create_labels() function

In [ ]:
"""
OVERVIEW
Basically our LISA dataset contains around 44k images and a bunch of labels

Labels are:
  - light status: {go, slow, stop}
  - bounding box of where the stoplight is xyxy coordinates (top left, bottom right)
  - each image may have many labels since there are many stoplights in a single image

DaySequence
"""

In [ ]:
"""
END HERE
"""

In [ ]:
# # centralize both annotations and images (not scattered in different folders)
# annotations = pd.DataFrame()

# target = "frameAnnotationsBOX.csv"
# annotation_paths = glob.glob(f"{path}/**/{target}", recursive=True)

# # I am going to omit the approximately 1000 pictures and labels in sample-dayClip6
# # and sample-nightClip1 because it is really annoying to work with right now and and
# # we have 43k other values we can work with first
# for p in annotation_paths:
#   tokens = p.split("/")
#   if tokens[-2] == "sample-nightClip1" or tokens[-2] == "sample-dayClip6":  # omit here
#     continue

#   # combine the csv files into one
#   new_frame = pd.read_csv(p, sep=";")
#   new_frame = new_frame.drop(['Origin file', 'Origin frame number', 'Origin track', 'Origin track frame number'], axis=1)
#   annotations = pd.concat([annotations, new_frame])

#   # move all of the nested image files into /content/images_dir
#   if len(tokens)==14:
#     subpath = path+"/"+tokens[-3]+"/"+tokens[-3]+"/"+tokens[-2]+"/frames"
#   else:
#     subpath = path+"/"+tokens[-2]+"/"+tokens[-2]+"/frames"
#   !cp -r {subpath}/* /content/images_dir

# filenames = annotations[annotations.columns[0]].str.split("/").str[-1]
# annotations["Filename"] = filenames
# annotations  # NOTE: our labels are in this df here

In [ ]:
# #TODO: define custom dataset class
# class TrafficLightDataset(Dataset):
#   def __init__(self, annotations_df, img_dir, transform=None, target_transform=None):
#     self.img_labels = annotations_df
#     self.img_dir = img_dir
#     self.transform = transform
#     self.target_transform = target_transform

#   def __len__(self):
#     return len(self.img_labels)

#   def __getitem__(self, idx):
#     img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
#     image = read_image(img_path)
#     label = tuple(self.img_labels.iloc[idx, 1:])
#     if self.transform:
#       image = self.transform(image)
#     if self.target_transform:
#       pass  # not sure how this will work on a tuple (worry about it later)
#     return image, label


In [ ]:
# #TODO: split our data into train and test dataloaders

# img_dir = "content/images_dir"
# dataset = TrafficLightDataset(annotations, img_dir, transform=None, target_transform=None)

# # shuffle our indices before splitting (need day and night in both sets)
# indices = torch.randperm(len(dataset))
# train_size = int(0.8*len(dataset))
# train_indices = indices[:train_size]
# test_indices = indices[train_size:]

# # split into train and test
# train_dataset = torch.utils.data.Subset(dataset, train_indices)
# test_dataset = torch.utils.data.Subset(dataset, test_indices)

# # we have our dataloaders here
# train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
# test_dataloader = DataLoader(test_dataset, batch_size=1000, shuffle=True)